In [11]:
import os

In [12]:
if not os.path.exists(path_res + 'roi_node'):
    os.mkdir(path_res + 'roi_node')

In [3]:
path_res = '../results/'
os.path.exists(path_res)

True

In [4]:
path_data = "/cobrain/groups/ml_group/data/HCP/cleaned_data/"

In [5]:
os.path.exists(path_data)

True

In [22]:
import numpy as np
import pandas as pd
import pickle

import os

from sklearn.preprocessing import StandardScaler, MinMaxScaler


from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.metrics import r2_score, make_scorer


path_data = "/cobrain/groups/ml_group/data/HCP/cleaned_data/"

with open(path_data + "mean_mesh_labels", 'rb') as f:
    mean_labels = pickle.load(f)


with open(path_data + "subjects_log_jac", 'rb') as f:
    log_jac = pickle.load(f)

with open(path_data + "subjects_thinkness", 'rb') as f:
    thinkness = pickle.load(f)

with open(path_data + "subjects_mesh_area", 'rb') as f:
    mesh_area = pickle.load(f)


targets_name = ['clustering', 'rich_club', 'betweenness', 'closeness',  'degree_centrality', 'eigenvector']

targets_data = []
for name in targets_name:
    with open(path_data + name, 'rb') as f:
        targets_data  += [pickle.load(f)]

print(np.array(targets_data).shape)


cv = ShuffleSplit(test_size=0.2)



elastic_param = {'alpha': [1e-10, 1e-7, 1e-5, 1e-3, 0.1, 1.0, 5, 10, 20, 50, 70, 100,], 
                  'l1_ratio': [1e-10, 1e-7, 1e-3, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                'max_iter':[1000,10000]}




path_res = '/home/ayagoz/connec/results/'
                   


thinkness = thinkness.reshape(789,-1)
log_jac = log_jac.reshape(789,-1)
mesh_area = mesh_area.reshape(789,-1)

X = np.concatenate([thinkness, log_jac, mesh_area], axis = -1)






cv = ShuffleSplit(test_size=0.2)

from sklearn.model_selection import cross_val_score

elastic = ElasticNet()


for tr, ts in cv.split(X):
    X_train, X_test = X[tr], X[ts]
    y_train, y_test = targets_data[0][tr], targets_data[0][ts]
    elastic = ElasticNet()
    elastic.fit(X_train, y_train)
    y_pred = elastic.predict(X_test)
    print(r2_score(y_test, y_pred))

(6, 789, 68)


In [ ]:
i = 0
for tr, ts in cv.split(X):
    X_train, X_test = X[tr], X[ts]
    y_train, y_test = targets_data[0][tr], targets_data[0][ts]
    print(i)
    i +=1
    #elastic = ElasticNet()

0


In [7]:
import numpy as np
import pandas as pd
import pickle

import os

from sklearn.preprocessing import StandardScaler, MinMaxScaler


from utils import get_all_attr

from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV, ShuffleSplit



path_data = "/cobrain/groups/ml_group/data/HCP/cleaned_data/"

with open(path_data + "mean_mesh_labels", 'rb') as f:
    mean_labels = pickle.load(f)

with open(path_data + "normed_connectomes", 'rb') as f:
    Y = pickle.load(f)

with open(path_data + "subjects_log_jac", 'rb') as f:
    log_jac = pickle.load(f)

with open(path_data + "subjects_thinkness", 'rb') as f:
    thinkness = pickle.load(f)

with open(path_data + "subjects_mesh_area", 'rb') as f:
    mesh_area = pickle.load(f)


    
thinkness = thinkness.reshape(789,-1)
log_jac = log_jac.reshape(789,-1)
mesh_area = mesh_area.reshape(789,-1)



cv = ShuffleSplit(test_size=0.2)



elastic_param = {'alpha': [1e-10, 1e-7, 1e-5, 1e-3, 0.1, 1.0, 5, 10, 20, 50, 70, 100,], 
                  'l1_ratio': [1e-10, 1e-7, 1e-3, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                'max_iter':[1000,10000]}




path_res = '../results/'

X = get_all_attr(thinkness, log_jac, mesh_area, mean_labels)

del thinkness
del log_jac
del mesh_area
del mean_labels

idx_nodes = list(range(1,4)) + list(range(5,39)) + list(range(40,71))
idx_nodes = np.array(idx_nodes)
print(idx_nodes.shape)
for i in range(68):
    for j in range(i+1,68):
        if np.sum(Y[:,i,j]) != 0:
            node1 = idx_nodes[i]
            node2 = idx_nodes[j]
            print(node1, node2)
            print(i,j)
            

            elastic = ElasticNet()
            StanS = StandardScaler()
            MinM = MinMaxScaler()
            cv = ShuffleSplit(test_size=0.2)
            rg = GridSearchCV(elastic, elastic_param, scoring = 'r2', 
                                    n_jobs=-1, cv = 10)

            rg.fit(X, Y[:,i,j])
            orig = pd.DataFrame.from_dict(rg.cv_results_)
            orig = orig.sort_values(by = 'rank_test_score').iloc[:1]

            rg.fit(StanS.fit_transform(X), Y[:,i,j])
            st = pd.DataFrame.from_dict(rg.cv_results_)
            st = st.sort_values(by = 'rank_test_score').iloc[:1]

            rg.fit(MinM.fit_transform(X), Y[:,i,j])
            mm = pd.DataFrame.from_dict(rg.cv_results_)
            mm = mm.sort_values(by = 'rank_test_score').iloc[:1]

            name_dir = 'exper_elastic_all_meshes_to_edge'
            edge = '/edge[' + str(i) + ',' + str(j) + ']'
            if not os.path.exists(path_res + name_dir):
                os.mkdir(path_res + name_dir)

            if not os.path.exists(path_res + name_dir + edge):
                os.mkdir(path_res + name_dir + edge)
            orig.to_csv(path_res + name_dir + edge +'/results_orig')
            st.to_csv(path_res + name_dir + edge + '/results_st')
            mm.to_csv(path_res + name_dir + edge + '/results_mm')


(68,)
1 2
0 1
1 3
0 2
1 5
0 3
1 7
0 5
1 8
0 6
1 9
0 7
1 10
0 8
1 11
0 9
1 12
0 10
1 13
0 11
1 14
0 12
1 15
0 13
1 16
0 14
1 17
0 15
1 18
0 16
1 19
0 17
1 20
0 18
1 21
0 19
1 22
0 20
1 23
0 21
1 24
0 22
1 25
0 23
1 26
0 24
1 27
0 25
1 28
0 26
1 29
0 27
1 30
0 28
1 31
0 29
1 32
0 30
1 33
0 31
1 34
0 32
1 35
0 33
1 36
0 34
1 37
0 35
1 38
0 36
1 40
0 37
1 42
0 39
1 43
0 40
1 44
0 41
1 45
0 42
1 46
0 43
1 47
0 44
1 48
0 45
1 49
0 46
1 50
0 47
1 51
0 48
1 52
0 49
1 56
0 53
1 57
0 54
1 58
0 55
1 59
0 56
1 60
0 57
1 62
0 59
1 63
0 60
1 64
0 61
1 65
0 62
1 66
0 63
1 70
0 67
2 3
1 2
2 5
1 3
2 6
1 4
2 7
1 5
2 8
1 6
2 9
1 7
2 10
1 8
2 11
1 9
2 12
1 10
2 13
1 11
2 14
1 12
2 15
1 13
2 16
1 14
2 17
1 15
2 18
1 16
2 19
1 17
2 20
1 18
2 21
1 19
2 22
1 20
2 23
1 21
2 24
1 22
2 25
1 23
2 26
1 24
2 27
1 25
2 28
1 26
2 29
1 27
2 30
1 28
2 31
1 29
2 32
1 30
2 33
1 31
2 34
1 32
2 35
1 33
2 36
1 34
2 37
1 35
2 38
1 36
2 40
1 37
2 42
1 39
2 43
1 40
2 44
1 41
2 45
1 42
2 46
1 43
2 47
1 44
2 48
1 45
2 49
1 46
2 

16 40
14 37
16 42
14 39
16 43
14 40
16 44
14 41
16 45
14 42
16 46
14 43
16 47
14 44
16 48
14 45
16 49
14 46
16 50
14 47
16 51
14 48
16 52
14 49
16 56
14 53
16 57
14 54
16 58
14 55
16 59
14 56
16 60
14 57
16 61
14 58
16 62
14 59
16 63
14 60
16 64
14 61
16 65
14 62
16 66
14 63
16 70
14 67
17 18
15 16
17 19
15 17
17 20
15 18
17 21
15 19
17 22
15 20
17 23
15 21
17 24
15 22
17 25
15 23
17 26
15 24
17 27
15 25
17 28
15 26
17 29
15 27
17 30
15 28
17 31
15 29
17 32
15 30
17 34
15 32
17 35
15 33
17 36
15 34
17 37
15 35
17 38
15 36
17 40
15 37
17 42
15 39
17 43
15 40
17 44
15 41
17 45
15 42
17 46
15 43
17 47
15 44
17 48
15 45
17 49
15 46
17 50
15 47
17 51
15 48
17 52
15 49
17 53
15 50
17 54
15 51
17 55
15 52
17 56
15 53
17 57
15 54
17 58
15 55
17 59
15 56
17 60
15 57
17 61
15 58
17 62
15 59
17 63
15 60
17 64
15 61
17 65
15 62
17 66
15 63
17 67
15 64
17 69
15 66
17 70
15 67
18 19
16 17
18 20
16 18
18 21
16 19
18 22
16 20
18 23
16 21
18 24
16 22
18 25
16 23
18 26
16 24
18 27
16 25
18 28
16 26
18 2

36 48
34 45
36 49
34 46
36 50
34 47
36 51
34 48
36 52
34 49
36 53
34 50
36 54
34 51
36 55
34 52
36 56
34 53
36 57
34 54
36 58
34 55
36 59
34 56
36 60
34 57
36 61
34 58
36 62
34 59
36 63
34 60
36 64
34 61
36 65
34 62
36 66
34 63
36 67
34 64
36 68
34 65
36 69
34 66
36 70
34 67
37 38
35 36
37 40
35 37
37 42
35 39
37 43
35 40
37 44
35 41
37 45
35 42
37 46
35 43
37 47
35 44
37 48
35 45
37 49
35 46
37 50
35 47
37 51
35 48
37 52
35 49
37 53
35 50
37 54
35 51
37 55
35 52
37 56
35 53
37 57
35 54
37 58
35 55
37 59
35 56
37 60
35 57
37 61
35 58
37 62
35 59
37 63
35 60
37 64
35 61
37 65
35 62
37 66
35 63
37 67
35 64
37 68
35 65
37 69
35 66
37 70
35 67
38 40
36 37
38 42
36 39
38 43
36 40
38 44
36 41
38 45
36 42
38 46
36 43
38 47
36 44
38 48
36 45
38 49
36 46
38 50
36 47
38 51
36 48
38 52
36 49
38 53
36 50
38 54
36 51
38 55
36 52
38 56
36 53
38 57
36 54
38 58
36 55
38 59
36 56
38 60
36 57
38 61
36 58
38 62
36 59
38 63
36 60
38 64
36 61
38 65
36 62
38 66
36 63
38 67
36 64
38 68
36 65
38 69
36 66
38 7

In [8]:
data = pd.DataFrame(columns = ['smg', 'the'])

In [9]:
data.to_csv(path_res + name_dir + edge +'/results_orig')

In [10]:
path_res + name_dir + edge +'/results_orig'

'../results/exper_elastic_all_meshes_to_edge/edge[66,67]/results_orig'